# Практическое задание 2 (часть 2)

# Распознавание именованных сущностей из Twitter с помощью LSTM

## курс "Математические методы анализа текстов"


### ФИО: Власов Андрей Валерьевич

## Введение

### Постановка задачи

В этом задании вы будете использовать рекуррентные нейронные сети для решения проблемы распознавания именованных сущностей (NER). Примерами именованных сущностей являются имена людей, названия организаций, адреса и т.д. В этом задании вы будете работать с данными twitter.

Например, вы хотите извлечь имена и названия организаций. Тогда для текста

    Yan Goodfellow works for Google Brain

модель должна извлечь следующую последовательность:

    B-PER I-PER    O     O   B-ORG  I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система введена, чтобы различать последовательные именованные сущности одного типа.

Решение этого задания будет основано на нейронных сетях, а именно на Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Numpy](http://www.numpy.org).
 
### Данные

Все данные содержатся в папке `./data`: `./data/train.txt`, `./data/validation.txt`, `./data/test.txt`.

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1s1rFOFMZTBqtJuQDcIvW-8djA78iUDcx)

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

## Часть 1. Подготовка данных (2 балла)

### Загрузка данных

Мы будем работать с данными, которые содержат твиты с тегами именованных сущностей. Каждая строка файла содержит пару токен(слово или пунктуация) и тег, разделенные пробелом. Различные твиты разделены пустой строкой.

Функция *read_data* считывает корпус из *file_path* и возвращает два списка: один с токенами и один с соответствующими токенам тегами. Также она заменяет все ники (токены, которые начинаются на символ *@*) на токен `<USR>` и url-ы (токены, которые начинаются на *http://* или *https://*) на токен `<URL>`. 

Вам необходимо реализовать эту функцию.

In [2]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_toks = []
    tweet_tags = []
    for line in open(file_path,'r', encoding='utf-8'):
        line = line.strip().split(' ')
        if line==['']:
            if tweet_toks != ['']:
                tokens.append(tweet_toks)
                tags.append(tweet_tags)
            tweet_toks = []
            tweet_tags = []
        else:
            if line[0].startswith('@'):
                line[0] = '<USR>'
            elif line[0].startswith('http://') or line[0].startswith('https://'):
                line[0] = '<URL>'
            tweet_toks.append(line[0])
            tweet_tags.append(line[1])

    return tokens, tags

Теперь мы можем загрузить 3 части данных:
 - *train* для тренировки модели;
 - *validation* для валидации и подбора гиперпараметров;
 - *test* для финального тестирования.

In [3]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [4]:
len(train_tokens),len(validation_tokens),len(test_tokens)

(5795, 724, 724)

Всегда полезно знать, с какими данными вы работаете. Выведем небольшую часть.

In [5]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения.

- {token}$\to${token id}: устанавливает соответствие между токеном и строкой в embedding матрице;
- {tag}$\to${tag id}: one hot encoding тегов.


Теперь вам необходимо реализовать функцию *build_dict*, которая должна возвращать словарь {token or tag}$\to${index} и контейнер, задающий обратное отображение.

In [6]:
from collections import defaultdict

In [7]:
def build_dict(tokens_or_tags, special_tokens):
    """
    tokens_or_tags: a list of lists of tokens or tags
    special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0

    cnt = 0
    
    for el in special_tokens:
        tok2idx[el] = cnt
        cnt += 1
        idx2tok.append(el)
    
    for tokens in tokens_or_tags:
        for token in tokens:
            if token not in tok2idx:
                tok2idx[token] = cnt
                cnt += 1
                idx2tok.append(token)    
    return tok2idx, idx2tok

После реализации функции *build_dict* вы можете создать словари для токенов и тегов. В нашем случае специальными токенами будут:
 - `<UNK>` токен для обозначаения слов, которых нет в словаре;
 - `<PAD>` токен для дополнения предложений одного батча до одинаковой длины.

In [8]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

### Генератор батчей

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины. Для некоторых фреймворков (таких как tensorflow) это необходимо сделать до подачи батча в нейронную сеть. В случае с pytorch это можно сделать как вне архитектуры нейронной сети, так и внутри. Мы выбрали более универсальный вариант и наш генератор батчей дополняет все последовательности внутри одного батча до одной длины.

Генератор батчей разбивает последовательность входных предложений и тегов на батчи размера batch_size. Размер последнего батча может быть меньше, если allow_smaller_last_batch is True, иначе последний батч исключается из генератора. Если включён параметр shuffle, данные перед разделением на батчи будут перемешаны. 

In [9]:
def batches_generator(batch_size, tokens_idxs, tags_idxs,
                      shuffle=True, allow_smaller_last_batch=True, device='cpu'):
    """
    Generates padded batches of tags_idxs and tags_idxs.
    
    batch_size : int, number of objects in one batch
    tokens_idxs : list of list of int
    tags_idxs : list of list of int
    shuffle : bool
    allow_smaller_last_batch : bool
    device: str, cpu or cuda:x
    
    yield x, y: torch.LongTensor and torch.LongTensor
    x - batch of tokens_idxs, y - batch of tags_idxs
    """
    n_samples = len(tokens_idxs)
    
    # Shuffle data if shuffle is True.
    # Don't modify the original tokens and tags!
    
    if shuffle:
        permutes = np.random.permutation(n_samples)
    else:
        permutes = np.arange(n_samples)
        
    # Get the number of batches
    
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches =  n_samples // batch_size +1
    else:
        n_batches =  n_samples // batch_size
        
    # For each k yield pair x and y
    for k in range(n_batches):
        start_idx = k*batch_size
        end_idx = min((k + 1) * batch_size, n_samples)
        
        cur_size = end_idx - start_idx
        
        x_ = []
        y_ = []
        mx_ln_tkns = 0
        for idx in permutes[start_idx: end_idx]:
            x_.append([token2idx[word] for word in tokens_idxs[idx]])
            y_.append([tag2idx[tag] for tag in tags_idxs[idx]])
            mx_ln_tkns = max(mx_ln_tkns, len(tags_idxs[idx]))
        
        x = np.ones([cur_size, mx_ln_tkns], dtype=np.long) * token2idx['<PAD>']
        y = np.ones([cur_size, mx_ln_tkns], dtype=np.long) * tag2idx['O']
        
        for n in range(cur_size):
            x[n, :len(x_[n])] = x_[n]
            y[n, :len(x_[n])] = y_[n]
              
        
        x = torch.from_numpy(x).long().to(device)
        y = torch.from_numpy(y).long().to(device)
        
        yield x, y

Протестируйте ваш генератор батчей:

In [10]:
test_nonrandom_batch_generator = batches_generator(
    batch_size=3,
    tokens_idxs=train_tokens[:7],
    tags_idxs=train_tags[:7],
    shuffle=False,
    allow_smaller_last_batch=True
)

batch_lengths = [3, 3, 1]
sequence_lengths = [26, 25, 8]
some_pad_tensor = torch.LongTensor([token2idx['<PAD>']] * 12)
some_outside_tensor = torch.LongTensor([tag2idx['O'] * 12])

for i, (tokens_batch, tags_batch) in enumerate(test_nonrandom_batch_generator):
    assert tokens_batch.dtype == torch.int64, 'tokens_batch is not LongTensor'
    assert tags_batch.dtype == torch.int64, 'tags_batch is not LongTensor'
    
    assert len(tokens_batch) == batch_lengths[i], 'wrong batch length'
    
    for one_token_sequence in tokens_batch:
        assert len(one_token_sequence) == sequence_lengths[i], 'wrong length of sequence in batch'
    
    if i == 0:
        assert torch.all(tokens_batch[2][-12:] == some_pad_tensor), "wrong padding"       
        assert torch.all(tags_batch[2][-12:] == some_outside_tensor), "wrong O tag"

## Часть 2. BiLSTM (3 балла)

Определите архитектуру сети, используя библиотеку pytorch. 

**Замечания:**
1. Для улучшения качества сети предлагается использовать дополнительный Embedding слой на входе (каждому слову ставится в соответствие обучаемый вектор). 

2. Не забудьте, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь.

In [11]:
class BiLSTMModel(torch.nn.Module):
    def __init__(self, vocabulary_size, n_tags, PAD_index,
                 embedding_dim, rnn_hidden_size,
                 dropout_zeroed_probability,
                 device='cpu'):
        '''
        Defines neural network structure.
        
        architecture: input -> Embedding -> BiLSTM -> Dropout -> Linear
        optimizer: Adam
        
        ----------
        Parameters
        
        vocabulary_size: int, number of words in vocabulary.
        n_tags: int, number of tags.
        PAD_index: int, index of padding character. Used for loss masking.
        embedding_dim: int, dimension of words' embeddings.
        rnn_hidden_size: int, number of hidden units in each LSTM cell
        dropout_zeroed_probability: float, dropout zeroed probability for Dropout layer.
        device: str, cpu or cuda:x
        '''
        super(BiLSTMModel, self).__init__()
        self.device = device
        self.rnn_hidden_size = rnn_hidden_size
        self.n_tags = n_tags
        self.embed = nn.Embedding(vocabulary_size, embedding_dim,PAD_index)
        self.lstm = nn.LSTM(embedding_dim, rnn_hidden_size, num_layers = 2,
                            batch_first=True, bidirectional=True,dropout=dropout_zeroed_probability)
        self.linear = nn.Linear(rnn_hidden_size*2, n_tags) # 2 for bidirection
        
    def forward(self, x_batch):
        '''
        Makes forward pass.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        # Embed word ids to vectors
        x_batch = self.embed(x_batch)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x_batch)
        
        out = self.linear(out)
        
        return out.to(self.device)

        
        
    def predict_for_batch(self, x_batch):
        '''
        Returns predictions for x_batch.
        
        return type: torch.LongTensor
        return shape: (number of samples in batch, number words in sentence.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        return nn.functional.softmax(self.forward(x_batch), dim=2).argmax(dim=2)
        
    
    def train_on_batch(self, x_batch, y_batch, optimizer, loss_function,clipping=False):
        '''
        Trains model on the given batch.
        
        ----------
        Parameters
        x_batch: np.ndarray with shape (number of samples in batch, number words in sentence).
        y_batch: np.ndarray with shape (number of samples in batch).
        optimizer: torch.optimizer class
        loss_function: torch loss class
        '''
         # zero out the gradients
        optimizer.zero_grad()
        
        y_pred = self.forward(x_batch).view(-1,self.n_tags)
        
        
        # calculate the loss
        loss = loss_function(y_pred, y_batch.view(-1))
        
        # calculate the gradients
        loss.backward()
        
        
        
        # update the parameters of the model
        optimizer.step()
        
        return loss

                


In [12]:
batch_size = 32
learning_rate = 1e-3
dropout_zeroed_probability = 0.8
embedding_dim = 200
rnn_hidden_size = 200


In [13]:
vocab_size = len(idx2token)
n_tags = len(idx2tag)
PAD_index = token2idx['<PAD>']
from tqdm import tqdm_notebook

In [303]:
bi_nn = BiLSTMModel(vocab_size, n_tags, PAD_index,
                 embedding_dim, rnn_hidden_size,
                 dropout_zeroed_probability,)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(bi_nn.parameters(), lr=learning_rate)

# number of epoch to train
n_epoch = 15
for epoch in tqdm_notebook(range(n_epoch)):
    learning_rate/= 1.1
    optimizer = torch.optim.Adam(bi_nn.parameters(), lr=learning_rate)
    for batch_num, (X_batch, y_batch) in enumerate(batches_generator(batch_size, train_tokens, train_tags)):
        bi_nn.train_on_batch(X_batch, y_batch, optimizer, loss_function) 
        
    with torch.no_grad():
        print('epoch =',epoch)
        print('Train')
        eval_conll(bi_nn, train_tokens, train_tags)
        print('Validation')
        eval_conll(bi_nn, validation_tokens, validation_tags)


epoch = 0 loss= tensor(24.3753, grad_fn=<AddBackward0>)
Train
processed 105778 tokens with 4489 phrases; found: 3 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; F1:  0.00

Validation
processed 12836 tokens with 537 phrases; found: 0 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; F1:  0.00

epoch = 1 loss= tensor(24.3753, grad_fn=<AddBackward0>)
Train
processed 105778 tokens with 4489 phrases; found: 342 phrases; correct: 68.

precision:  19.88%; recall:  1.51%; F1:  2.82

Validation
processed 12836 tokens with 537 phrases; found: 31 phrases; correct: 1.

precision:  3.23%; recall:  0.19%; F1:  0.35

epoch = 2 loss= tensor(24.3753, grad_fn=<AddBackward0>)
Train
processed 105778 tokens with 4489 phrases; found: 969 phrases; correct: 266.

precision:  27.45%; recall:  5.93%; F1:  9.75

Validation
processed 12836 tokens with 537 phrases; found: 92 phrases; correct: 16.

precision:  17.39%; recall:  2.98%; F1:  5.09

epoch = 3 loss= tensor(24.3753, grad_fn=<AddBackward0>

In [278]:
learning_rate

0.0002393920493691634

Для тестирования сети мы подготовили для вас две функции:
 - *predict_tags*: получает батч данных и трансформирует его в список из токенов и предсказанных тегов;
 - *eval_conll*: вычисляет метрики precision, recall и F1

In [14]:
from evaluation_ner import precision_recall_f1

In [15]:
def predict_tags(model, token_idxs_batch):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(token_idxs_batch)
#     print(tag_idxs_batch)
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            if token_idx != token2idx['<PAD>']:
#                 print(tag_idx)
                tags.append(idx2tag[tag_idx])
                tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, x_batch)
        ground_truth_tags = [idx2tag[tag_idx] for tag_idx in y_batch[0]]

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(tags_batch[0] + ['O'])
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results


In [269]:
with torch.no_grad():
    eval_conll(bi_nn, test_tokens, test_tags)

processed 13258 tokens with 604 phrases; found: 983 phrases; correct: 176.

precision:  17.90%; recall:  29.14%; F1:  22.18



In [274]:
with torch.no_grad():
    eval_conll(bi_nn, test_tokens, test_tags)

processed 13258 tokens with 604 phrases; found: 788 phrases; correct: 182.

precision:  23.10%; recall:  30.13%; F1:  26.15



In [283]:
with torch.no_grad():
    eval_conll(bi_nn, test_tokens, test_tags)

processed 13258 tokens with 604 phrases; found: 739 phrases; correct: 153.

precision:  20.70%; recall:  25.33%; F1:  22.78



### Эксперименты

Задайте BiLSTMModel. Рекомендуемые параметры:
- *batch_size*: 32;
- начальное значение *learning_rate*: 0.01-0.001
- *dropout_zeroed_probability*: 0.7-0.9
- *embedding_dim*: 100-200
- *rnn_hidden_size*: 150-200

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1 мере на валидационной и тестовой выборках было не меньше 0.35. 

Если сеть плохо обучается, попробуйте использовать следующие модификации:
    * используйте gradient clipping
    * на каждой итерации уменьшайте learning rate (например, в 1.1 раз)
    * попробуйте вместо Adam другие оптимизаторы 
    * экспериментируйте с dropout

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

In [307]:
batch_size = 32
learning_rate = 1e-3
dropout_zeroed_probability = 0.8
embedding_dim = 200
rnn_hidden_size = 200

bi_nn = BiLSTMModel(vocab_size, n_tags, PAD_index,
                 embedding_dim, rnn_hidden_size,
                 dropout_zeroed_probability)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(bi_nn.parameters(), lr=learning_rate)


In [310]:
# number of epoch to train
n_epoch = 20
for epoch in tqdm_notebook(range(n_epoch)):
    
    losses = []
    loss = 0
    if epoch % 4 == 0:
        learning_rate/=5
    else:
        learning_rate/= 1.1
    optimizer = torch.optim.Adam(bi_nn.parameters(), lr=learning_rate)
    for batch_num, (X_batch, y_batch) in enumerate(batches_generator(batch_size, train_tokens, train_tags)):
        loss += bi_nn.train_on_batch(X_batch, y_batch, optimizer, loss_function)
        bi_nn.embed.weight.data[PAD_index] = 0
    losses.append(loss/batch_num)
    with torch.no_grad():
        print('epoch =',epoch,'loss =',losses[-1])
        print('Test')
        eval_conll(bi_nn, test_tokens, test_tags)
        print('Validation')
        eval_conll(bi_nn, validation_tokens, validation_tags)
    nn.utils.clip_grad_norm_(bi_nn.parameters(), 0.5)

epoch = 0 loss = tensor(0.2089, grad_fn=<DivBackward0>)
Test
processed 13258 tokens with 604 phrases; found: 23 phrases; correct: 4.

precision:  17.39%; recall:  0.66%; F1:  1.28

Validation
processed 12836 tokens with 537 phrases; found: 18 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; F1:  0.00

epoch = 1 loss = tensor(0.2106, grad_fn=<DivBackward0>)
Test
processed 13258 tokens with 604 phrases; found: 22 phrases; correct: 4.

precision:  18.18%; recall:  0.66%; F1:  1.28

Validation
processed 12836 tokens with 537 phrases; found: 39 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; F1:  0.00

epoch = 2 loss = tensor(0.2113, grad_fn=<DivBackward0>)
Test
processed 13258 tokens with 604 phrases; found: 26 phrases; correct: 5.

precision:  19.23%; recall:  0.83%; F1:  1.59

Validation
processed 12836 tokens with 537 phrases; found: 25 phrases; correct: 1.

precision:  4.00%; recall:  0.19%; F1:  0.36

epoch = 3 loss = tensor(0.2096, grad_fn=<DivBackward0>)
Test
proces

KeyboardInterrupt: 

In [311]:
with torch.no_grad():
    eval_conll(bi_nn, test_tokens, test_tags)

processed 13258 tokens with 604 phrases; found: 27 phrases; correct: 5.

precision:  18.52%; recall:  0.83%; F1:  1.58



In [21]:
batch_size = 32
learning_rate = 1e-2
dropout_zeroed_probability = 0.8
embedding_dim = 200
rnn_hidden_size = 150

bi_nn = BiLSTMModel(vocab_size, n_tags, PAD_index,
                 embedding_dim, rnn_hidden_size,
                 dropout_zeroed_probability)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(bi_nn.parameters(), lr=learning_rate)


In [22]:
# number of epoch to train
n_epoch = 20
for epoch in tqdm_notebook(range(n_epoch)):
    
    losses = []
    loss = 0
    learning_rate/= 1.1
    optimizer = torch.optim.Adam(bi_nn.parameters(), lr=learning_rate)
    bi_nn.train(True)
    for batch_num, (X_batch, y_batch) in enumerate(batches_generator(batch_size, train_tokens, train_tags)):
        
        loss += bi_nn.train_on_batch(X_batch, y_batch, optimizer, loss_function)
#         bi_nn.embed.weight.data[PAD_index] = 0
    losses.append(loss/batch_num)
    
    bi_nn.train(False)
    with torch.no_grad():
        print('epoch =',epoch,'loss =',losses[-1])
        print('Test')
        eval_conll(bi_nn, test_tokens, test_tags)
        print('Validation')
        eval_conll(bi_nn, validation_tokens, validation_tags)
    nn.utils.clip_grad_norm_(bi_nn.parameters(), 0.75)

epoch = 0 loss = tensor(0.2669, grad_fn=<DivBackward0>)
Test
processed 13258 tokens with 604 phrases; found: 195 phrases; correct: 50.

precision:  25.64%; recall:  8.28%; F1:  12.52

Validation
processed 12836 tokens with 537 phrases; found: 75 phrases; correct: 26.

precision:  34.67%; recall:  4.84%; F1:  8.50

epoch = 1 loss = tensor(0.1636, grad_fn=<DivBackward0>)
Test
processed 13258 tokens with 604 phrases; found: 876 phrases; correct: 124.

precision:  14.16%; recall:  20.53%; F1:  16.76

Validation
processed 12836 tokens with 537 phrases; found: 335 phrases; correct: 83.

precision:  24.78%; recall:  15.46%; F1:  19.04

epoch = 2 loss = tensor(0.1291, grad_fn=<DivBackward0>)
Test
processed 13258 tokens with 604 phrases; found: 1415 phrases; correct: 198.

precision:  13.99%; recall:  32.78%; F1:  19.61

Validation
processed 12836 tokens with 537 phrases; found: 563 phrases; correct: 138.

precision:  24.51%; recall:  25.70%; F1:  25.09

epoch = 3 loss = tensor(0.1063, grad_fn=

In [ ]:
with torch.no_grad():
    eval_conll(bi_nn, test_tokens, test_tags)

 - [X] используйте gradient clipping (пробовал разные значения, ксожалению лучше не сделало, хотя пробовал в разных местах, мб неверно применяю, но посмотрел здесь поэтому по логике должно быть норм https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/language_model/main.py#L81)
 - [X] на каждой итерации уменьшайте learning rate (например, в 1.1 раз) (помогает улучшать минимум на 5 %)
 - [X] попробуйте вместо Adam другие оптимизаторы (Nesterov & AdaDelta не дали улучшенияк Adam)
 - [X] экспериментируйте с dropout (менял значения с 0.7-0.9, наилучшее 0.8)
 
Как видим последняя модель, дала наилучшее качество на тесте > 30%.(best)

К сожалению, она начинает переобучаться, что видно из колебаний значения F1 на test & validation).

Архитектура очень чувствительна к настройке гиперпараметров, тк что немного отходит от best качество сразу проваливается. И увы только за 30 минут до конца делайна удалось установить, что lr =1e-2 дает лучше качество, тч не успел поделать тестов с ней.

Также сделал собственоручное обнуление для embedding[PAD_index], взял с этого сайта https://discuss.pytorch.org/t/ignore-a-specific-index-of-embedding/12590